In [1]:
# num of nodes
n = 8
A = [[0,1], [1,2], [0,3], [3,4], [3,6], [3,7], [4,2], [4,5], [5,2]]

In [2]:
# convert array of edges to adjecency matrix

M = []
for i in range(n):
    M.append([0]*n)
    
for u, v in A:
    M[u][v] = 1

M

[[0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [3]:
# convert array of edges to adjecency list
from collections import defaultdict

D = defaultdict(list)

for u, v in A:
    D[u].append(v)

D


defaultdict(list, {0: [1, 3], 1: [2], 3: [4, 6, 7], 4: [2, 5], 5: [2]})

In [4]:
import networkx as nx

G = nx.Graph()
G.add_node('A2')
G.add_nodes_from(['A3','A4','A5'])

G.add_edge('A2', 'A3')
G.add_edges_from([('A3','A4'),('A4','A5')])

for i in range(2, 6):
    G.add_edge(f'B{i}', f'C{i}')
    if 2<i<5:
        G.add_edge(f'B{i}', f'B{i+1}')
    if i<5:
        G.add_edge(f'C{i}', f'C{i+1}')

In [5]:
print(G.number_of_nodes(), 'nodes')
print(G.number_of_edges(), 'edges')
print(list(G['C3']))
print(list(G.edges('C3')))

12 nodes
12 edges
['C2', 'B3', 'C4']
[('C3', 'C2'), ('C3', 'B3'), ('C3', 'C4')]


In [6]:
from collections import deque
import heapq

class Node:
    def __init__(self, val, next=None) -> None:
        self.next = next
        self.value = val
        
class Stack:
    def __init__(self) -> None:
        self.top = None
    
    def is_empty(self):
        return self.top is None
    
    def push(self, val):
        self.top = Node(val=val, next=self.top)
        
    def pop(self):
        if self.is_empty():
            raise RuntimeError('Stack is empty!')
        
        val = self.top.value
        self.top = self.top.next
        return val
    
def dfs_search(G, src):
    seen = set()
    node_from = {}
    
    stack = Stack()
    stack.push(src)
    
    while not stack.is_empty():
        curr_node = stack.pop()
        
        for node in list(G[curr_node]):
            if node not in seen:
                node_from[node] = curr_node
                stack.push(node)
                seen.add(node)
    
    return node_from

def bff_search(G, src):
    seen = set()
    node_from = {}
    
    queue = deque()
    queue.appendleft(src)
    
    while not len(queue) <= 0:
        curr_node = queue.pop()
        
        for node in list(G[curr_node]):
            if node not in seen:
                node_from[node] = curr_node
                queue.appendleft(node)
                seen.add(node)
                
    return node_from

def path_to(node_from, src, target):
    if not target in node_from:
        raise ValueError('Unreachable')
    
    path = []
    curr_node = target
    
    while curr_node!=src:
        path.append(curr_node)
        curr_node=node_from[curr_node]
    
    path.append(src)
    path.reverse()
    return path

In [7]:
import random

G = nx.Graph()

G.add_nodes_from(range(20))

for _ in range(30):
    node1 = random.randint(0, 19)
    node2 = random.randint(0, 19)
    if node1 != node2:
        G.add_edge(node1, node2)

In [8]:
node_from = bff_search(G, 10)
print(node_from)
path = path_to(node_from, 10, 19)
print(path)

{13: 10, 10: 13, 3: 13, 5: 13, 1: 13, 19: 3, 11: 3, 9: 5, 7: 5, 0: 19, 16: 9, 15: 9, 6: 9, 4: 0, 14: 0, 17: 0, 18: 14, 2: 14}
[10, 13, 3, 19]


In [17]:
import networkx as nx


G = nx.DiGraph()

G.add_edges_from([(1,2), (1,3), (3,4), (3,5), (5,6), (7,8)])


    
def dfs_search(G, src):
    seen = set()
    node_from = {}
    
    stack = Stack()
    stack.push(src)
    
    while not stack.is_empty():
        curr_node = stack.pop()
        
        for node in list(G[curr_node]):
            if node not in seen:
                node_from[node] = curr_node
                stack.push(node)
                seen.add(node)
    
    return node_from

In [22]:
list(nx.neighbors(G, 2))

[]

In [38]:
g_cycle = nx.DiGraph()
g_cycle.add_edges_from([(1,2), (2,3), (1,3), (4,2), (3,4)])


def has_cycle_2squared(graph):
    
    for node in g_cycle:
        this_run_seen = set()
        stack = Stack()
        
        stack.push(node)
        
        while not stack.is_empty():
            curr_node = stack.pop()
            
            for n in nx.neighbors(graph, curr_node):
                if n == node:
                    return True
                elif n not in this_run_seen:
                    this_run_seen.add(n)
                    stack.push(n)
    return False


def has_cycle(graph):
    def dfs(node, temp_seen, perm_seen):
        if node in perm_seen:
            return False
        if node in temp_seen:
            return True
        
        temp_seen.add(node)
        for successor in graph.successors(node):
            if dfs(successor, temp_seen, perm_seen):
                return True
        
        temp_seen.remove(node)
        perm_seen.add(node)
        return False
    
    temp_seen = set()
    perm_seen = set() 
    for node in graph.nodes():
        if node not in perm_seen:
            if dfs(node, temp_seen, perm_seen):
                return True
    
    return False
                
has_cycle(g_cycle)

True

In [41]:
g_cycle = nx.DiGraph()
g_cycle.add_edges_from([(1,2), (2,3), (1,3), (4,2), (4,3)])

def topological_sort(graph):
    seen = set()
    postorder = []
    
    def dfs(node):
        seen.add(node)
        for successor in graph.successors(node):
            if successor not in seen:
                dfs(successor)
        postorder.append(node)
    
    for node in graph.nodes():
        if node not in seen:
            dfs(node)
    
    return reversed(postorder)

list(topological_sort(g_cycle))

[4, 1, 2, 3]

In [43]:
import random

G = nx.DiGraph()

# Add 20 nodes
G.add_nodes_from(range(1, 21))

# Randomly add edges with weights
for _ in range(40):  # Add 40 random edges
    u = random.randint(1, 20)
    v = random.randint(1, 20)
    if u != v:  # Avoid self-loops
        weight = round(random.uniform(1.0, 10.0), 2)  # Random weight between 1.0 and 10.0
        G.add_edge(u, v, weight=weight)

In [56]:
import heapq

pq = []
heapq.heappush(pq, (50, 'Z'))
heapq.heappush(pq, (20, 'B'))
heapq.heappush(pq, (10, 'A'))

heapq.heappop(pq)

(10, 'A')

In [ ]:
def dijkstra(graph, src):
    dist = 0
    prev = 1
    visit = 2
    
    nodes_info = {src: [0]}